# Coursera capstone

### Project description:


### Project structure:
- extract from foursquare all the pizza places in the indian city of Surat
- extract from foursquare all the indian restaurants in the italian city of Milan
- improve both the dataframe eliminating non useful data
- create a map with all the entries in it

- compare

In [2]:
#import libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize

import folium # map rendering library

from folium.plugins import MarkerCluster

### Retrieving coordinates of both cities

In [3]:
address1 = 'Surat, India'
address2 = 'Milan, Italy'

geolocator = Nominatim()
location1 = geolocator.geocode(address1)
location2 = geolocator.geocode(address2)

latitude1 = location1.latitude
latitude2 = location2.latitude

longitude1 = location1.longitude
longitude2 = location2. longitude

print('The geograpical coordinate of Surat are {}, {}.'.format(latitude1, longitude1))
print('The geograpical coordinate of Milan are {}, {}.'.format(latitude2, longitude2))

C:\Users\albta\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Surat are 21.1864607, 72.8081281.
The geograpical coordinate of Milan are 45.4668, 9.1905.


## Query request from foursquare

With the foursquare api it is possible to retrieve all the venues of pizza places in Surat

In [4]:
CLIENT_ID = 'R53KA41YY4PR1GFMFNJPB4SZ1XTNEBPPCBXFJTGTZBK1TLVT' # your Foursquare ID
CLIENT_SECRET = 'DCRG2AJM2W1YXTARXXQZISLLFFULS2JCOBN4RQRGUPGCV4FP' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R53KA41YY4PR1GFMFNJPB4SZ1XTNEBPPCBXFJTGTZBK1TLVT
CLIENT_SECRET:DCRG2AJM2W1YXTARXXQZISLLFFULS2JCOBN4RQRGUPGCV4FP


In [5]:
search_query1 = 'Pizza'
radius = 5000
print(search_query1 + ' .... OK!')

Pizza .... OK!


In [6]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query1, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=R53KA41YY4PR1GFMFNJPB4SZ1XTNEBPPCBXFJTGTZBK1TLVT&client_secret=DCRG2AJM2W1YXTARXXQZISLLFFULS2JCOBN4RQRGUPGCV4FP&ll=21.1864607,72.8081281&v=20180605&query=Pizza&radius=5000&limit=100'

#### Retrieve the places from Milan

Now we do the same, but changing the query to "indian" in order to find all the indian restaurants in Milan

In [7]:
search_query2 = 'Indian'

url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius, LIMIT)


#### Transform the result in a json file

In [8]:
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()

In [9]:
# assign relevant part of JSON to venues
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe1 = pd.json_normalize(venues1)
dataframe2 = pd.json_normalize(venues2)

dataframe1.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,512ae1a2e4b0e1bba52e0555,US PIZZA,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591110173,False,hp petrol pump,opp. science center,21.169030,72.794801,"[{'label': 'display', 'lat': 21.16902982216688...",2383,395007,IN,Sūrat,Gujarāt,India,"[hp petrol pump (opp. science center), Sūrat 3..."
1,56fbd1dd498e4b42f400effc,Pizza Hut,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591110173,False,"OB/1, Tribhuvan Complex, Next to Baskin Robbins",Ghod Dod Road,21.214119,72.826786,"[{'label': 'display', 'lat': 21.214119, 'lng':...",3637,395007,IN,Sūrat,Gujarāt,India,"[OB/1, Tribhuvan Complex, Next to Baskin Robbi..."
2,4cc3f8e7b2beb1f7ea271a4c,Pizza Hut,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591110173,False,Ghod Dhod Road,NaN,21.174736,72.801362,"[{'label': 'display', 'lat': 21.17473587564860...",1482,NaN,IN,Sūrat,Gujarāt,India,"[Ghod Dhod Road, Sūrat, Gujarāt, India]"
3,4cdfa307825e721efc3a7245,Domino's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591110173,False,Amand mahal road,NaN,21.194000,72.799000,"[{'label': 'display', 'lat': 21.194, 'lng': 72...",1265,395010,IN,Sūrat,Gujarāt,India,"[Amand mahal road, Sūrat 395010, Gujarāt, India]"
4,4d667e6dc2ccb60c48a370ac,Pizza Papa Johns,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591110173,False,Fame Raj Empire,NaN,21.172083,72.816262,"[{'label': 'display', 'lat': 21.17208261625433...",1809,NaN,IN,Sūrat,Gujarāt,India,"[Fame Raj Empire, Sūrat, Gujarāt, India]"


In [10]:
dataframe2.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.crossStreet,venuePage.id,location.neighborhood
0,4c2bad8f57a9c9b66b4af667,Indian Snack's & Kabab,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1591110174,False,Via Pollaiuolo 2,45.488664,9.187391,"[{'label': 'display', 'lat': 45.48866362501622...",2445,IT,Milano,Lombardia,Italia,"[Via Pollaiuolo 2, Milano Lombardia, Italia]",NaN,NaN,NaN,NaN
1,4cd57d25b2a4370449507e44,Indian Tattoo Family,"[{'id': '4bf58dd8d48988d1de931735', 'name': 'T...",v-1591110174,False,Via Arcimboldi 5,45.461063,9.188190,"[{'label': 'display', 'lat': 45.461063, 'lng':...",663,IT,Milano,Lombardia,Italia,"[Via Arcimboldi 5, Milano Lombardia, Italia]",NaN,NaN,NaN,NaN
2,4b05887af964a52036c822e3,Indian Cafè,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591110174,False,97/99 Corso Garibaldi,45.477187,9.185467,"[{'label': 'display', 'lat': 45.47718714702632...",1221,IT,Milano,Lombardia,Italia,"[97/99 Corso Garibaldi, 20011 Milano Lombardia...",20011,NaN,NaN,NaN
3,52ff91a211d208d8fffe0d14,indian,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591110174,False,NaN,45.473067,9.212583,"[{'label': 'display', 'lat': 45.473067, 'lng':...",1859,IT,NaN,NaN,Italia,[Italia],NaN,NaN,NaN,NaN
4,4d30c2ee2748b60cc7cedea0,Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591110174,False,Via castaldi,45.476368,9.202305,"[{'label': 'display', 'lat': 45.47636835908737...",1408,IT,Milano,Lombardia,Italia,"[Via castaldi, Milano Lombardia, Italia]",NaN,NaN,NaN,NaN


In [11]:
# change the name of latitude and longitude columns to improve readability
dataframe1.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)
dataframe2.rename(columns={"location.lat": "lat", "location.lng": "lng"}, inplace=True)

## Create two maps with all the entries

We create the maps of the two cities using the folium library

In [12]:
#map of surat
map_surat = folium.Map(location=[latitude1, longitude1], zoom_start=12)
map_milan = folium.Map(location=[latitude2, longitude2], zoom_start=12)

In [13]:
#add the entries to the map of surat


for lat, lng in zip(dataframe1['lat'], dataframe1['lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(map_surat)
    
map_surat

In [14]:
#add the entries to the map of milan


for lat, lng in zip(dataframe2['lat'], dataframe2['lng']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(map_milan)
    
map_milan

These probably are not all the entries because in Surat many bars that does not includes "pizza" in their name use to sell pizza, on the other side in Milan it is possible that not all indian restaurant includes "indian" in their name.

### We can now count the city with more places

In [15]:
dataframe1['id'].count()

36

In [16]:
dataframe2['id'].count()

41

Using the cluster function we can create cluster of the zones with more entries in the same zone

In [17]:
marker_cluster = MarkerCluster().add_to(map_surat)

locations1 = dataframe1[['lat', 'lng']]
locationlist1 = locations1.values.tolist()

for point in range(0, len(locationlist1)):
    folium.Marker(locationlist1[point], popup=dataframe1['name']).add_to(marker_cluster)

map_surat

In [18]:
marker_cluster1 = MarkerCluster().add_to(map_milan)

locations2 = dataframe2[['lat', 'lng']]
locationlist2 = locations2.values.tolist()

for point in range(0, len(locationlist2)):
    folium.Marker(locationlist2[point], popup=dataframe1['name']).add_to(marker_cluster1)

map_milan

## Conclusion

Now giving a look to the cluster we can tell the areas with the less places and where would be nice to open a pizza place/indian restaurant, those are represented by the green markers on the map:

- Surat: the best zones are all the peripheral areas around the city center, like the nortern and the western part of the city
- Milan: similar to Surat, the best zones are the peripheral ones respect to the city center (but obviously we should also check the neighbourhood, and the distance from important places to determine a potential success)